## Personen-Analyse

### Vorbereiten der Daten

In [ ]:
# Importieren der benötigten Bibliotheken: 
import pandas as pd
from lxml import etree
import unicodedata
import plotly.express as px
import numpy as np

In [ ]:
# Laden der MARC-xml-Datei in ElementTree und umwandeln in eine Liste an Datensätzen: 
tree = etree.parse('dataset_tutorial_eco.xml')
root = tree.getroot()
ns = {'marc': 'http://www.loc.gov/MARC21/slim'}
records = root.findall('.//marc:record', namespaces=ns)
print("Gefundene Records:", len(records))

In [ ]:
# Funktion zum Extrahieren von Datensätzen
def parse_record(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    
    def extract_text(xpath_query):
        fields = record.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
        return "unknown"

    idn = extract_text("marc:controlfield[@tag='001']")
    titel = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    author = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    author_rela = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='e']")
    add_author = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='a']")
    add_author_rela = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='e']")
    jahr = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='c']")

    return {
        "idn": idn,
        "titel": titel,
        "author": author,
        "author_rela": author_rela,
        "additional_author": add_author,
        "added_rela": add_author_rela,
        "jahr": jahr        
    }

In [ ]:
# Übergabe der einzelnen Datensätze an die Funktion "parse_record":
result = [parse_record(record) for record in records]
df = pd.DataFrame(result)
df

### Analyse

In [ ]:
# Häufigkeit der Autoren zählen und top 25 ausgeben: 
author_counts = df['author'].value_counts().head(25).reset_index()
author_counts.columns=['author', 'count']
author_counts

In [ ]:
# Balkendiagramm erstellen
fig1 = px.bar(author_counts, x='author', y='count', height=550,
             title='Häufigste Personen im Datenset', color='count', color_continuous_scale='Viridis')
fig1.show()

In [ ]:
# Suche im Dataframe nach einer bestimmten Person:

df.loc[df['author'] == 'Tamm, Gerrit']

Weiterführend:   
https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/workshops/2024_Metadaten_Analyse/02_DNBLab_MARC_Analyse_Personen.ipynb